In [5]:
!qbraid configure magic


Successfully configured qBraid IPython magic commands.

You can now use the qBraid-CLI from inside a Jupyter notebook as follows:

        In [1]: %load_ext qbraid_magic

        In [2]: %qbraid



In [7]:
!git init

Reinitialized existing Git repository in /home/jovyan/.git/


In [ ]:
!git commit -m "Initial Commit"

In [1]:
#Daequan Peele | #July 2025
#Attempt to perform VQE with pure MoS2 (using xyz files?)

#qiskit version 1.4.3
#qiskit-algorithms version 0.3.1 
#qiskit Aer version 0.17.1
#PySCF version 2.9.0
#qiskit-nature 0.7.2
#numpy version latest
#ase version latest

In [2]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import StatevectorEstimator 

from qiskit_nature.second_q.hamiltonians import LatticeModel
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.hamiltonians.lattices import Lattice, BoundaryCondition
from qiskit.circuit.library import TwoLocal

import qiskit_nature
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms.optimizers import SPSA, SLSQP, COBYLA 

qiskit_nature.settings.use_pauli_sum_op = False  # pylint: disable=undefined-variable
# pylint: enable=line-too-long
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
import numpy as numpy
from qiskit.circuit.library import EfficientSU2 
from qiskit_aer.primitives import Estimator

#creating pseudo quantum instance from IBM qauntum provider/create quantum instance 
#qiskit-ibm-runtime version 0.40.1 required
from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import GenericBackendV2

from pyscf import gto, scf, cc
from pyscf.pbc import gto
from pyscf.pbc.gto.cell import fromfile

import ase
from ase.io import read
from ase.neighborlist import NeighborList

In [9]:
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=10)
noiseless_estimator = Estimator(approximation=True)

#Ground state energy for MoS2 attempt

molecule = MoleculeInfo(
    #Angstorm coords
    symbols=["H","H"],
    coords=([0.0,0.0, -0.3625],[0.0, 0.0, 0.3625]),
    multiplicity=1,
    charge=0,
) 


driver = PySCFDriver.from_molecule(molecule)
problem = driver.run()

second_q_ops = problem.second_q_ops() #2nd Quantization of Operator
num_spatial_orbitals = problem.num_spatial_orbitals 
num_particles = problem.num_particles

mapper = ParityMapper(num_particles=num_particles) #Set Mapper 

hamiltonian = second_q_ops[0] #Set Hamiltonian 

#2 qubit reduction 
qubit_op = mapper.map(hamiltonian)
device = GenericBackendV2(num_qubits=5)
coupling_map = device.coupling_map
noise_model = NoiseModel.from_backend(device)
noisy_estimator = Estimator(
    backend_options={"coupling_map": coupling_map, "noise_model": noise_model}
)

result = exact_solver(qubit_op, problem)
exact_energies.append(result.total_energies)

print("Exact Result:", result.total_energies)
optimizer = SPSA(maxiter=100)
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")

vqe = VQE(noisy_estimator, var_form, optimizer)
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
vqe_result = problem.interpret(vqe_calc).total_energies

print("VQE Result:", vqe_result)

Exact Result: [-1.13722138]
VQE Result: [-1.11860477]


In [24]:
#FAILS
#Attempts to use the PySCF driver for a vqe and a MoS2 Cell object, fails when passing information into the PySCFDriver class
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=10)
noiseless_estimator = Estimator(approximation=True)

#Ground state energy for pure MoS2 attempt (via xyz file)

#molecule = gto.M(atom = "pure_MoS2.xyz")

cell = gto.Cell()
cell.symbols = ["Mo", "S", "S"]
cell.a = [[3.16192779,  0.00001556,  0.00000000],
          [1.58092805,  2.73825609,  0.00000000],
          [0.00000000,  0.00000000,  50.00000000]]
cell.coords = cell.a
cell.units = 'Angstrom'
cell.atom = '''
            Mo     1.58095372         0.91283272         2.58110226 
            S     3.16190300         1.82547670         1.04217642  
            S     3.16190300         1.82547669         4.12002810  
            '''
cell.pseudo = 'gth-pade'
cell.basis = 'sto3g'
cell.build()
#molecule.tofile('purestMoS2.xyz')


driver = PySCFDriver.from_molecule(cell.fromfile('purestMoS2.xyz'))
#QiskitNatureError: 'Failed to build the PySCF Molecule object.'
#makes sense, a Cell object is trying to go into the from_moluecule() function that expects a MoleculeInfo object. 
problem = driver.run()

second_q_ops = problem.second_q_ops() #2nd Quantization of Operator
num_spatial_orbitals = problem.num_spatial_orbitals 
num_particles = problem.num_particles

mapper = ParityMapper(num_particles=num_particles) #Set Mapper 

hamiltonian = second_q_ops[0] #Set Hamiltonian 

#2 qubit reduction 
qubit_op = mapper.map(hamiltonian)
device = GenericBackendV2(num_qubits=5)
coupling_map = device.coupling_map
noise_model = NoiseModel.from_backend(device)
noisy_estimator = Estimator(
    backend_options={"coupling_map": coupling_map, "noise_model": noise_model}
)

result = exact_solver(qubit_op, problem)
exact_energies.append(result.total_energies)

print("Exact Result:", result.total_energies)
optimizer = SPSA(maxiter=100)
var_form = EfficientSU2(qubit_op.num_qubits, entanglement="linear")

vqe = VQE(noisy_estimator, var_form, optimizer)
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
vqe_result = problem.interpret(vqe_calc).total_energies

print("VQE Result:", vqe_result)

<class 'pyscf.pbc.gto.cell.Cell'> does not have attributes  symbols coords units


QiskitNatureError: 'Failed to build the PySCF Molecule object.'

In [29]:
#1. Use ase to laod atomic data (.xyz)

atoms = read("purestMoS2.xyz")
print("Cell Vectors:\n", atoms.cell)
print("Atomic Positions:\n", atoms.positions) 

#ASE passes prewritten perodic boundary conditions to the Qiskit Nature wrapper later

#2. Create a neighbor list and determine hopping behavior 
cutoff = 2.6 #distance between which to atoms are considered neighbors, measured in Angstorms
cutoffs = [cutoff] * len(atoms) #decides which pairs of sites are edges in the lattice, and which get hopping parameters in the Hubbard Hamiltonian

nl = NeighborList(cutoffs=cutoffs, self_interaction=False, bothways=True)
nl.update(atoms)

weighted_edges = []

#example hopping strength, is modular
hopping_strength = -1.0 #eV, decides how strongly electrons can tunnel between neighboring sites, as it relates to Hubbard Hamiltonians
#MoS2 (and other TMDs) tend to range between -0.5 to -2.5 eV
for i in range(len(atoms)):
    neighbors, offsets = nl.get_neighbors(i)
    for j, offset in zip(neighbors, offsets): 
        #avoid dupes, only add if i < j 
        if i < j:
            weighted_edges.append((i, j, hopping_strength))
print("Weighted edges:\n", weighted_edges)

#3. Create Qiskit Nature Lattice Object 

num_nodes = len(atoms)
lattice = Lattice.from_nodes_and_edges(num_nodes, weighted_edges) 

print(lattice) #confirms that the lattice object formed properly and exists in memory 

#Using FermiHubbardModel to create a tight-binding model 
#Parameters:

#lattice object
U = 2.0 #Onsite_interaction (Coulomb Repulsion)

#seperate optional parameters: 
#chemical potential, 
#spin potential,
#

model = FermiHubbardModel(lattice, U)

#4. Turn lattice into hamiltonian 
hamiltonian = model.second_q_op()

#5. Map to qubits 
mapper = JordanWignerMapper()
qubit_op = mapper.map(hamiltonian)
print("Qubit Operators:\n", qubit_op)

#6. Set up and solve with VQE 
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz", reps=2) #TwoLocal covers more variational space as compared to the initial Hartree Fock state
optimizer = COBYLA(maxiter=600) #Constained Optimization by Linear Approximations, does NOT use gradient information to optimize, robust to noise
estimator = Estimator()

vqe = VQE(ansatz=ansatz, optimizer=optimizer, estimator=estimator)
#vqe = VQE(ansatz, optimizer, estimator) <- does not work, causes error in COBYLA object
result = vqe.compute_minimum_eigenvalue(qubit_op)

#Exact result for comparison
exact_solver = NumPyMinimumEigensolver()
exact_result = exact_solver.compute_minimum_eigenvalue(qubit_op)

print("VQE ground state energy:", result.eigenvalue.real)
print("Exact ground state energy:", exact_result.eigenvalue.real)


Cell Vectors:
 Cell([[3.16192779, 1.556e-05, 0.0], [1.58092805, 2.73825609, 0.0], [0.0, 0.0, 50.0]])
Atomic Positions:
 [[1.58095372 0.91283272 2.58110226]
 [3.161903   1.8254767  1.04217642]
 [3.161903   1.82547669 4.1200281 ]]
Weighted edges:
 [(0, np.int64(1), -1.0), (0, np.int64(2), -1.0), (0, np.int64(1), -1.0), (0, np.int64(2), -1.0), (0, np.int64(1), -1.0), (0, np.int64(2), -1.0), (0, np.int64(1), -1.0), (0, np.int64(2), -1.0), (0, np.int64(1), -1.0), (0, np.int64(2), -1.0), (0, np.int64(1), -1.0), (0, np.int64(2), -1.0), (0, np.int64(1), -1.0), (0, np.int64(1), -1.0), (0, np.int64(1), -1.0), (0, np.int64(1), -1.0), (0, np.int64(1), -1.0), (0, np.int64(1), -1.0), (0, np.int64(2), -1.0), (0, np.int64(2), -1.0), (0, np.int64(2), -1.0), (0, np.int64(2), -1.0), (0, np.int64(2), -1.0), (0, np.int64(2), -1.0), (1, np.int64(2), -1.0), (1, np.int64(2), -1.0), (1, np.int64(2), -1.0), (1, np.int64(2), -1.0), (1, np.int64(2), -1.0), (1, np.int64(2), -1.0), (1, np.int64(2), -1.0)]
Qubit Ope